In [1]:
import pandas as pd
import sys
sys.path.append('/home/vinas/bedtools')
import pybedtools
from pybedtools import BedTool

In [2]:
data_dir = '/mlbio_scratch/vinas/chip-atlas'
pybedtools.set_tempdir('/mlbio_scratch/vinas/tmp')

#### Load metadata

In [3]:
ncols = 15
experiment_df = pd.read_csv(f'{data_dir}/experimentList.tab',
                            delimiter='\t',
                            header=None,
                            usecols=list(range(ncols))) # on_bad_lines='skip',
# Ref: https://github.com/inutano/chip-atlas/wiki#experimentList_schema
experiment_df.columns = ['exp_id', 'genome', 'track_type_class', 'track_type',
                         'cell_type_class', 'cell_type', 'cell_type_description',
                         'processing_logs', 'processing_logs_bisulfite_seq',
                         'title', 'metadata_1', 'metadata_2', 'metadata_3', 'metadata_4', 'metadata_5']
mouse_experiment_df = experiment_df[experiment_df['genome'] == 'mm10']
df = mouse_experiment_df[mouse_experiment_df['cell_type_class'].isin(['Pluripotent stem cell'])]
df = df[df['track_type_class'] == 'TFs and others']
df

,exp_id,genome,track_type_class,track_type,cell_type_class,cell_type,cell_type_description,processing_logs,processing_logs_bisulfite_seq,title,metadata_1,metadata_2,metadata_3,metadata_4,metadata_5
434383,DRX000343,mm10,TFs and others,Chd2,Pluripotent stem cell,ES cells,NaN,"3576947,95.1,9.9,93",ES CHD2 ChIP-Seq 20100705 Elution,sample_name=DRS000311,strain=129/Ola,sample comment=mouse EB5 cell lines medium con...,cell_line=EB5,NaN,NaN
434557,DRX013314,mm10,TFs and others,Kdm1a,Pluripotent stem cell,EpiSC,NaN,"104699053,85.6,38.5,19216",Epi-Lsd1,sample_name=DRS013077,sample comment=EpiSC cells were cultured on mu...,NaN,NaN,NaN,NaN
434562,DRX013319,mm10,TFs and others,Rest,Pluripotent stem cell,EpiSC,NaN,"72899748,36.2,83.5,17692",Epi-REST,sample_name=DRS013082,sample comment=EpiSC cells were cultured on mu...,NaN,NaN,NaN,NaN
434563,DRX013320,mm10,TFs and others,Rcor2,Pluripotent stem cell,EpiSC,NaN,"70560067,33.8,83.3,14412",Epi-COREST,sample_name=DRS013083,sample comment=EpiSC cells were cultured on mu...,NaN,NaN,NaN,NaN
434564,DRX013321,mm10,TFs and others,Ehmt2,Pluripotent stem cell,EpiSC,NaN,"56839755,41.2,77.9,13087",Epi-G9a,sample_name=DRS013084,sample comment=EpiSC cells were cultured on mu...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589026,SRX997158,mm10,TFs and others,Brd4,Pluripotent stem cell,ES cells,NaN,"25333917,98.0,48.8,6833",GSM1659409: WT Brd4 ChIP-seq Rep 2; Mus muscul...,source_name=WT Brd4 ChIP-seq,strain background=V6.5 (129SvJaexC57BL/6),cell type=Embryonic stem cells (ESCs),transfected with=H2A.Z(WT)-YFP,genotype/variation=expressing wild type H2A.Z,"chip antibody=Brd4 (Bethyl A301-985A50, lot 3)"
589027,SRX997159,mm10,TFs and others,Brd2,Pluripotent stem cell,ES cells,NaN,"25681235,98.0,47.7,20053",GSM1659410: K3R3 Brd2 ChIP-seq Rep 1; Mus musc...,source_name=K3R3 Brd2 ChIP-seq,strain background=V6.5 (129SvJaexC57BL/6),cell type=Embryonic stem cells (ESCs),transfected with=H2A.Z(K3R3)-YFP,genotype/variation=expressing a ubiquitin muta...,"chip antibody=Brd2 (Bethyl A302-583A, lot 1)"
589028,SRX997160,mm10,TFs and others,Brd2,Pluripotent stem cell,ES cells,NaN,"19240486,98.1,69.6,8589",GSM1659411: K3R3 Brd2 ChIP-seq Rep 2; Mus musc...,source_name=K3R3 Brd2 ChIP-seq,strain background=V6.5 (129SvJaexC57BL/6),cell type=Embryonic stem cells (ESCs),transfected with=H2A.Z(K3R3)-YFP,genotype/variation=expressing a ubiquitin muta...,"chip antibody=Brd2 (Bethyl A302-583A, lot 1)"
589029,SRX997161,mm10,TFs and others,Brd4,Pluripotent stem cell,ES cells,NaN,"24511162,97.3,57.8,7980",GSM1659412: K3R3 Brd4 ChIP-seq Rep 1; Mus musc...,source_name=K3R3 Brd4 ChIP-seq,strain background=V6.5 (129SvJaexC57BL/6),cell type=Embryonic stem cells (ESCs),transfected with=H2A.Z(K3R3)-YFP,genotype/variation=expressing a ubiquitin muta...,"chip antibody=Brd4 (Bethyl A301-985A50, lot 3)"


#### Filter relevant peaks

In [ ]:
chip = BedTool(f'{data_dir}/allPeaks_light.mm10.05.bed.gz')
unique_exp_ids = df['exp_id'].unique()
chip_filtered = chip.filter(lambda x: x[3] in unique_exp_ids)
chip_filtered = chip_filtered.saveas(f'{data_dir}/mouse_stem_peaks.mm10.05.bed')
chip_filtered

#### MM10 (ENSEMBL 96) genome

In [24]:
mm10_df = pd.read_csv(f'{data_dir}/Mus_musculus.ENS96.csv', index_col=0)
mm10_df['chr'] = 'chr'+mm10_df.index
mm10_df.to_csv(f'{data_dir}/Mus_musculus.ENS96.tsv', header=None, sep='\t')
mm10_df.set_index('chr')[['chromStart', 'chromEnd', 'gene_id']].to_csv(f'{data_dir}/Mus_musculus_3col.ENS96.tsv', header=None, sep='\t')

#### Intersect

In [70]:
chip_filtered = BedTool(f'{data_dir}/mouse_stem_embryo_peaks.mm10.05.bed')
genes = BedTool(f'{data_dir}/Mus_musculus_3col.ENS96.tsv')
mapped_genes = chip_filtered.intersect(genes, wo=True).moveto(f'{data_dir}/mm10_stem_cells_chip_atlas_network.bed')

#### Matched peaks

In [8]:
matched_df = pd.read_csv(f'{data_dir}/mm10_stem_cells_chip_atlas_network.bed', delimiter='\t', header=None)
matched_df.columns = ['chr_1', 'start_1', 'end_1', 'exp_id', 'MACS2', 'chr_2', 'start_2', 'end_2', 'TG', 'overlap']
matched_df = matched_df[matched_df['exp_id'].isin(df['exp_id'].unique())]
matched_df

,chr_1,start_1,end_1,exp_id,MACS2,chr_2,start_2,end_2,TG,overlap
0,chr1,3102101,3102502,SRX1410928,129,chr1,3102016,3102125,ENSMUSG00000064842,24
1,chr1,3206037,3206465,SRX7001433,430,chr1,3205901,3671498,ENSMUSG00000051951,428
2,chr1,3206082,3206420,SRX14812472,531,chr1,3205901,3671498,ENSMUSG00000051951,338
3,chr1,3206088,3206450,SRX14812473,439,chr1,3205901,3671498,ENSMUSG00000051951,362
4,chr1,3206103,3206433,SRX18990882,418,chr1,3205901,3671498,ENSMUSG00000051951,330
...,...,...,...,...,...,...,...,...,...,...
58202268,chrY,90843980,90844175,SRX3581851,119,chrY,90837413,90844040,ENSMUSG00000099871,60
58202269,chrY,90843989,90844072,SRX823726,88,chrY,90837413,90844040,ENSMUSG00000099871,51
58202270,chrY,90843990,90844039,SRX868174,89,chrY,90837413,90844040,ENSMUSG00000099871,49
58202271,chrY,90843990,90844062,SRX555895,97,chrY,90837413,90844040,ENSMUSG00000099871,50


In [23]:
df_ = df.set_index('exp_id').loc[matched_df['exp_id']]
matched_df['TF_name'] = df_['track_type'].values
matched_df['cell_type'] = df_['cell_type'].values

/tmp/ipykernel_1199932/3129011447.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_df['TF'] = df_['track_type'].values
/tmp/ipykernel_1199932/3129011447.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_df['cell_type'] = df_['cell_type'].values


In [37]:
genes_df = pd.read_csv(f'{data_dir}/Mus_musculus.ENS96.csv')
genes_df_ = genes_df.set_index('gene_id')
matched_df['TG_name'] = genes_df_.loc[matched_df['TG']]['gene_name'].values
matched_df

/tmp/ipykernel_1199932/2530099654.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_df['TG_name'] = genes_df_.loc[matched_df['TG']]['gene_name'].values


,chr_1,start_1,end_1,exp_id,MACS2,chr_2,start_2,end_2,TG,overlap,TF,cell_type,TG_name
0,chr1,3102101,3102502,SRX1410928,129,chr1,3102016,3102125,ENSMUSG00000064842,24,Pou3f1,EpiSC,Gm26206
1,chr1,3206037,3206465,SRX7001433,430,chr1,3205901,3671498,ENSMUSG00000051951,428,Trp53,ES cells,Xkr4
2,chr1,3206082,3206420,SRX14812472,531,chr1,3205901,3671498,ENSMUSG00000051951,338,Trim24,ES cells,Xkr4
3,chr1,3206088,3206450,SRX14812473,439,chr1,3205901,3671498,ENSMUSG00000051951,362,Trim24,ES cells,Xkr4
4,chr1,3206103,3206433,SRX18990882,418,chr1,3205901,3671498,ENSMUSG00000051951,330,Trp53,mESC derived neural cells,Xkr4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58202268,chrY,90843980,90844175,SRX3581851,119,chrY,90837413,90844040,ENSMUSG00000099871,60,Cbx3,ES cells,Gm21742
58202269,chrY,90843989,90844072,SRX823726,88,chrY,90837413,90844040,ENSMUSG00000099871,51,Aff3,ES cells,Gm21742
58202270,chrY,90843990,90844039,SRX868174,89,chrY,90837413,90844040,ENSMUSG00000099871,49,Fgfr1,ES cells,Gm21742
58202271,chrY,90843990,90844062,SRX555895,97,chrY,90837413,90844040,ENSMUSG00000099871,50,Dppa2,ES cells,Gm21742


In [45]:
matched_df.to_csv(f'{data_dir}/mm10_stem_cells_chip_atlas_network.csv')

In [68]:
matched_df['cell_type'].value_counts()

cell_type
ES cells                                      41213335
Embryoid Bodies                                2279598
iPS cells                                      1596013
mESCs, differentiated                          1483780
mESC derived neural cells                      1388218
EpiLC                                          1070463
EpiSC                                           656975
F9                                              555079
iPSC intermediates                              472510
PGCLC                                           327595
mESC derived erythroid cells                    283773
P19                                             279510
mESC derived haematopoietic progenitor          252373
mESC derived endodermal cells                   175280
mESC derived mesodermal cells                   164156
mESC derived haemogenic endothelium             132932
ZHBTc4                                          105961
Germline stem cell-like cells                    99583
